In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
df_news = pd.read_csv('data/Macquarie_19900101-20220420.csv')

In [4]:
df_news

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100601e+06,20220420,한국경제,김익환,대박 노리다 '700억' 물리더니 투자 스타일 확 바뀐 회사 [김익환의 컴퍼니워치],경제>금융_재테크,경제>증권_증시,경제>무역,NaN,NaN,NaN,"양홍석,양,이어룡,포메이션8,구본웅","미국,대주주,싱가포르,서울시,경기도","마음그룹,SC,LS그룹,예스코홀딩스,국민연,맥쿼리인프라,예스코,LS,대신증권,투자은행","700억,투자,스타일,회사,김익환,컴퍼니워치,LS,그룹,계열,도시가스,업체,예스코홀...","대신증권,예스코홀딩스,고배당주,맥쿼리인프라,도시가스,스타트업,부회장,700억,투자금...",LS그룹 계열의 도시가스업체 예스코홀딩스가 대신증권과 맥쿼리인프라를 비롯한 고배당주...,https://www.hankyung.com/economy/article/20220...,NaN
1,2.100101e+06,20220415,매일경제,신화,"미래에셋 리츠 ETF, 순자산 2000억 돌파",경제>증권_증시,경제>금융_재테크,경제>유통,NaN,NaN,NaN,NaN,NaN,"SK리츠,맥쿼리인프라,미래에셋,미래에셋자산운용,제이알글로벌리츠,ESR켄달스퀘어리츠,...","ETF,미래에셋,리츠,돌파,자산,미래에셋자산운용,TIGER,리츠부동산인프라,상장지수...","리츠부동산인프라,etf,2000억,원어치,미래에셋,인프라스트럭처,켄달스퀘어리츠,인컴...",미래에셋자산운용이 'TIGER 리츠부동산인프라' 상장지수펀드(ETF)의 순자산이 2...,http://news.mk.co.kr/newsRead.php?no=339384&ye...,NaN
2,1.100101e+06,20220414,경향신문,강연주 기자 play@kyunghyang.com,"김대기, 사외 감독이사 맡아 최소 6억 수령 ‘관피아’ 논란",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,이명박,김대기,권오인,한덕수,이창양",NaN,"경향신문,맥쿼리인프라,김앤장,두산중공업,SK하이닉스,현대두산인프라코어,현대두산인프라...","김대기,사외,감독이사,수령,최소,관피아,논란,2015년,SK이노베이션,재직,한화생명...","내정자,사외이사,두산중공업,한화생명,맥쿼리인프라,두산인프라코어,sk이노베이션,공무원...",2015년부터 SK이노베이션 현대두산인프라코어 등서 재직\n한화생명에선 ‘고문’으로...,https://www.khan.co.kr/national/national-gener...,NaN
3,1.100101e+06,20220414,경향신문,강연주 기자 play@kyunghyang.com,"[단독]김대기, 민간 사외이사 지내며 ‘최소 6억4100만원’ 수령 ‘관피아’ 논란",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,이명박,김대기,권오인,한덕수,이창양",NaN,"경향신문,맥쿼리인프라,김앤장,두산중공업,경제정의실천연합,현대두산인프라코어,맥쿼리한국...","김대기,민간,사외,이사,최소,수령,관피아,논란,김대기,대통령,비서실장,내정자,201...","내정자,사외이사,두산중공업,한화생명,sk이노베이션,이사회,두산인프라코어,맥쿼리인프라...",김대기 대통령 비서실장 내정자가 2015년부터 현재까지 민간 기업 사외 감독이사 활...,https://www.khan.co.kr/national/national-gener...,NaN
4,1.101001e+06,20220414,한겨레,박종오 기자,"김대기 비서실장, 바로 그 ‘맥쿼리’ 이사였다",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,노무현,이명박,김대기,윤증현,한승수,7천만원","인천김포고속도로,종로구,광주시,서울,9호선,통의동,서울지하철,서울시,부산신항,광주","에스케이이노베이션,인수위,맥쿼리인프라,대한상사중재원,서라벌도시가스,맥쿼리,두산중공업...","김대기,비서실장,맥쿼리,이사,김대기,대통령,비서실장,내정자,13일,민자,사업,투자,...","내정자,맥쿼리,이명박,맥쿼리인프라,민자사업,사외이사,한화생명,민자,윤증현,에스케이이...",김대기 대통령 비서실장 내정자가 13일 현재 민자사업 투자를 주력으로 하는 맥쿼리한...,http://www.hani.co.kr/arti/politics/politics_g...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,1.100901e+06,20060221,중앙일보,NaN,[브리핑] 인프라 펀드 상장 규모 11조원 넘을 듯,경제>증권_증시,국제>유럽_EU,경제>금융_재테크,NaN,NaN,NaN,NaN,"미국,환산,한국,호주,런던","신한금융그룹,매쿼리은행,메릴린치,투자은행,MKIF","11조,인프라,펀드,상장,규모,투자은행,호주,최대,투자,은행,매쿼리은행,신한금융그룹...","1조,한국,다음달,런던,메릴린치,mkif,매쿼리,호주,11조,11억,gdr,신한금융...",호주 최대 투자은행인 매쿼리은행과 신한금융그룹이 공동 설립한 인프라 펀드의 한국.런...,https://news.joins.com/article/2210809,예외
1023,2.100601e+06,20060209,한국경제,NaN,"CLSA,""인프라펀드 상장 관련 건설주 주목""",경제>금융_재테크,경제>증권_증시,경제>산업_기업,NaN,NaN,NaN,NaN,NaN,"CL,정부,크레디리요네(CLSA)증권,한경닷컴,PF","CLSA,인프라펀드,상장,상장,건설,주목,크레디리요네,CLSA,증권,맥쿼리인프라펀드...","건설주,크레디리요네,대림산업,현대산업개발,기투자,맥쿼리인프라펀드,기록중,인프라펀드,...",9일 크레디리요네(CLSA)증권은 맥쿼리인프라펀드(MKIF) 상장 추진으로 사모펀드...,NaN,NaN
1024,2.100601e+06,20060207,한국경제,NaN,"맥쿼리 인프라펀드, 韓ㆍ英 동시상장 추진",경제>증권_증시,경제>산업_기업,경제>국제경제,NaN,NaN,NaN,NaN,"한국,영국,항만,증시,런던","금융감독원,맥쿼리한국인프,맥쿼리,라투융자회사,MKIF","맥쿼리,인프라,펀드,추진,韓ㆍ英,동시,상장,도로,항만,사회간접자본,SOC,시설,투자...","맥쿼리,mkif,영국,런던,인프라펀드,사회간접자본,라투,감독원,라투융자회사,한국인프...",도로 항만 등 사회간접자본(SOC)시설에 투자하는 인프라펀드인 맥쿼리한국인프\n라투...,NaN,NaN
1025,1.100901e+06,20060207,중앙일보,NaN,[브리핑] 신한-맥쿼리 은행 펀드 한 영서 상장,경제>금융_재테크,경제>증권_증시,경제>국제경제,NaN,NaN,NaN,NaN,"서울,천안,호주,런던,인천공항","신한금융그룹,금융감독원,맥쿼리 은행,맥쿼리,맥쿼리코리아,MKIF","신한,맥쿼리,은행,펀드,영서,상장,금융감독원,신한금융그룹,호주,최대,투자,은행,맥쿼...","고속도로,신한금융그룹,호주,맥쿼리코리아,mkif,인프라스트럭처,천안,논산,감독원,맥...",금융감독원은 7일 신한금융그룹과 호주 최대 투자 은행인 맥쿼리 은행이 공동 설립한 ...,https://news.joins.com/article/2137057,예외


In [5]:
df_news = df_news.iloc[:20]

In [6]:
pd.set_option('display.max_colwidth', None)
df_news['URL']

0                       https://www.hankyung.com/economy/article/202204208696i
1                        http://news.mk.co.kr/newsRead.php?no=339384&year=2022
2     https://www.khan.co.kr/national/national-general/article/202204142115005
3     https://www.khan.co.kr/national/national-general/article/202204141105001
4            http://www.hani.co.kr/arti/politics/politics_general/1038853.html
5                        http://news.mk.co.kr/newsRead.php?no=325079&year=2022
6                  https://www.donga.com/news/article/all/20220404/112693737/1
7                                 http://www.mk.co.kr/economy/view/2022/298364
8                        http://news.mk.co.kr/newsRead.php?no=292701&year=2022
9       https://www.khan.co.kr/economy/economy-general/article/202203311551001
10                       http://news.mk.co.kr/newsRead.php?no=290811&year=2022
11                       http://news.mk.co.kr/newsRead.php?no=236242&year=2022
12                      https://www.hankyung.com/fin

In [ ]:
# Loop through URLs, scrap information, identify the main body
# URL이 주어졌을 경우 해당 매체 웹사이트로부터 뉴스기사전문 크롤링

#최종결과물에 대한 텍스트 전처리-불용어 제거
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
    clean_text = re.sub(cleanr, '', raw_html)
    return clean_text


def crawling_main_text(url):
                
        #조선일보 - 미완성
        if 'chosun' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")
            content_obj = soup.find_all("div", class_="article-body")
            
            content = clean_html(str(content_obj))
             
        
        #중앙일보
        elif 'joins' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_body")
            content = clean_html(str(content_obj))
        
        
        #동아일보
        elif 'donga' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_txt")
            content = clean_html(str(content_obj))
        
        
        #한겨례신문
        elif 'hani' in url:
            url = urllib.request.Request(url)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
            
            content_obj = soup.find_all('div', class_="text")
            content = clean_html(str(content_obj))
        

        #경향신문
        elif 'khan' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('p', class_="content_text")
            content = clean_html(str(content_obj))
        
        
        #한국경제
        elif 'hankyung' in url:
            url = urllib.request.Request(url, headers={'User-Agent': 'Custom'})
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
    
            content_obj = soup.find_all('div', id="articletxt")
            content = clean_html(str(content_obj))
        
        
        #매일경제
        elif 'mk' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
            soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

            content_obj = soup_alt.find_all('div', class_="art_txt")

            content = clean_html(str(content_obj))
    
        return content
                                     

In [ ]:
#For Loop을 통한 본문 크롤링
#데이터 프레임에 추가

for url in df_news['URL']:
    print(url)
    main_content = crawling_main_text(url)
    print(main_content)
    df_news['main_content'] = main_content
    

In [ ]:
df_news

## Reference Code for Individual Media Source

### 한겨례

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

def contents_hankyoreh():
    
    #샘플 기사 URL
    list_url = "https://www.hani.co.kr/arti/politics/politics_general/1038853.html"
    
    url = urllib.request.Request(list_url)
    res = urllib.request.urlopen(url).read().decode("utf-8")
    #print(res)
    
    soup = BeautifulSoup(res, "html.parser")
    params1 = []
    params2 = []
    
    for link1, link2 in zip(soup.find_all('p', class_="date-time"), soup.find_all('div', class_="text")):
        params1.append(link1.get_text())
        params2.append(link2.get_text())
        
    for i1, i2 in zip(params1, params2):
        #print(i1.strip(), end=' ')
        print(i2.strip())
        
contents_hankyoreh()
    

### 한국경제

In [ ]:
#한국경제와 같은 경우 

import re
import urllib.request
from bs4 import BeautifulSoup

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def contents_hankyung():
    
    #샘플 기사 URL
    list_url = "https://www.hankyung.com/finance/article/2021122646691"
    
    url = urllib.request.Request(list_url, headers={'User-Agent': 'Custom'}) #
    res = urllib.request.urlopen(url).read().decode("utf-8")
    
    soup = BeautifulSoup(res, "html.parser")
    
    content_obj = soup.find_all('div', id="articletxt")
    content = cleanhtml(str(content_obj))
    return content
        

contents_hankyung()
    

### 경향신문

In [ ]:
#샘플 기사 URL
list_url = 'https://www.khan.co.kr/national/national-general/article/202204142115005'

def contents_kyunghyang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('p', class_="content_text")

    content = cleanhtml(str(content_obj))
    return content

contents_kyunghyang(list_url)

### 동아일보

In [ ]:
list_url = 'https://www.donga.com/news/article/all/20220404/112693737/1'

def contents_donga(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_txt")

    content = cleanhtml(str(content_obj))
    return content

contents_donga(list_url)

### 중앙일보

In [ ]:
list_url = 'https://news.joins.com/article/2137057'

def contents_joongang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_body")

    content = cleanhtml(str(content_obj))
    return content

contents_joongang(list_url)

### 매일경제

In [ ]:
import bs4
list_url = 'https://www.mk.co.kr/news/stock/view/2022/04/339384/'

def contents_maeil(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
    soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

    content_obj = soup_alt.find_all('div', class_="art_txt")

    content = cleanhtml(str(content_obj))
    return content

contents_maeil(list_url)

### 조선일보

In [ ]:
import re
from bs4 import BeautifulSoup

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content_obj = []

def contents_chosun(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")
    #content_obj = soup.find_all('section', class_="article-body")
    for tag in soup.select("div", class_="article-body"):
        print(tag)
        #write_ws.append([tag.text])
    
    content = cleanhtml(str(content_obj))
    return content

contents_chosun(list_url)

### 조선일보

In [14]:
import re
from bs4 import BeautifulSoup

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content_obj = []

def contents_chosun(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")
    #content_obj = soup.find_all('section', class_="article-body")
    for tag in soup.select("div", class_="article-body"):
        print(tag)
        #write_ws.append([tag.text])
    
    content = cleanhtml(str(content_obj))
    return content

contents_chosun(list_url)

<div id="fusion-app"><div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div></div>
<div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div>


'[]'